# RAPIDS cuDF

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ritchieng/deep-learning-wizard/blob/master/docs/machine_learning/gpu/rapids_cudf.ipynb)

## Environment Setup

### Check Version

#### Python Version

In [1]:
# Check Python Version
!python --version

Python 3.10.12


#### Ubuntu Version

In [2]:
# Check Ubuntu Version
!lsb_release -a

No LSB modules are available.
Distributor ID:	Ubuntu
Description:	Ubuntu 22.04.3 LTS
Release:	22.04
Codename:	jammy


#### Check CUDA Version

In [3]:
# Check CUDA/cuDNN Version
!nvcc -V && which nvcc

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0
/usr/local/cuda/bin/nvcc


#### Check GPU Version

In [4]:
# Check GPU
!nvidia-smi

Tue May 21 22:26:29 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   62C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

#Setup:
RAPIDS cuDF is now a native library in Colab.  While you can use the cuDF library independantly from Pandas to get the maximum GPU acceleration of your workflow, you can also use `cuDF.pandas` to let GPU accelerate your existing pandas workflow with zero-code change.  [You can check out the RAPIDS website and it's tutorial notebook for more details](https://rapids.ai/cudf-pandas/).

## Critical Imports

In [5]:
# Critical imports
import cudf
import os
import numpy as np
import pandas as pd

## Creating

### Create a Series of integers

In [6]:
gdf = cudf.Series([1, 2, 3, 4, 5, 6])
print(gdf)
print(type(gdf))

0    1
1    2
2    3
3    4
4    5
5    6
dtype: int64
<class 'cudf.core.series.Series'>


### Create a Series of floats

In [7]:
gdf = cudf.Series([1., 2., 3., 4., 5., 6.])
print(gdf)

0    1.0
1    2.0
2    3.0
3    4.0
4    5.0
5    6.0
dtype: float64


### Create a  Series of strings


In [8]:
gdf = cudf.Series(['a', 'b', 'c'])
print(gdf)

0    a
1    b
2    c
dtype: object


### Create 3 column DataFrame
- Consisting of dates, integers and floats

In [9]:
# Import
import datetime as dt

# Using a dictionary of key-value pairs
# Each key in the dictionary represents a category
# The key is the category's name
# The value is a list of the values in that category
gdf = cudf.DataFrame({
    # Create 10 busindates ess from 1st January 2019 via pandas
    'dates': pd.date_range('1/1/2019', periods=10, freq='B'),
    # Integers
    'integers': [i for i in range(10)],
    # Floats
    'floats': [float(i) for i in range(10)]
})

# Print dataframe
print(gdf)

       dates  integers  floats
0 2019-01-01         0     0.0
1 2019-01-02         1     1.0
2 2019-01-03         2     2.0
3 2019-01-04         3     3.0
4 2019-01-07         4     4.0
5 2019-01-08         5     5.0
6 2019-01-09         6     6.0
7 2019-01-10         7     7.0
8 2019-01-11         8     8.0
9 2019-01-14         9     9.0


### Create 2 column Dataframe
- Consisting of integers and string category

In [10]:
# Using a dictionary
# Each key in the dictionary represents a category
# The key is the category's name
# The value is a list of the values in that category
gdf = cudf.DataFrame({
    'integers': [1 ,2, 3, 4],
    'string': ['a', 'b', 'c', 'd']
})

print(gdf)

   integers string
0         1      a
1         2      b
2         3      c
3         4      d


### Create a 2 Column  Dataframe with Pandas Bridge
- Consisting of integers and string category
- For all string columns, you must convert them to type `category` for filtering functions to work intuitively (for now)

In [11]:
# Create pandas dataframe
pandas_df = pd.DataFrame({
    'integers': [1, 2, 3, 4],
    'strings': ['a', 'b', 'c', 'd']
})

# Convert string column to category format
pandas_df['strings'] = pandas_df['strings'].astype('category')

# Bridge from pandas to cudf
gdf = cudf.DataFrame.from_pandas(pandas_df)

# Print dataframe
print(gdf)

   integers strings
0         1       a
1         2       b
2         3       c
3         4       d


## Viewing

### Printing Column Names

In [12]:
gdf.columns

Index(['integers', 'strings'], dtype='object')

### Viewing Top of DataFrame

In [13]:
num_of_rows_to_view = 2
print(gdf.head(num_of_rows_to_view))

   integers strings
0         1       a
1         2       b


### Viewing Bottom of DataFrame

In [14]:
num_of_rows_to_view = 3
print(gdf.tail(num_of_rows_to_view))

   integers strings
1         2       b
2         3       c
3         4       d


## Filtering

### Method 1: Query

#### Filtering Integers/Floats by Column Values
- This only works for floats and integers, not for strings

In [15]:
print(gdf.query('integers == 1'))

   integers strings
0         1       a


#### Filtering Strings by Column Values
- This only works for floats and integers, not for strings so this will return an error!

In [16]:
print(gdf.query('strings == a'))

KeyError: 'a'

### Method 2:  Simple Columns

#### Filtering Strings by Column Values


In [17]:
# Filtering based on the string column
print(gdf[gdf.strings == 'b'])

   integers strings
1         2       b


#### Filtering Integers/Floats by Column Values

In [18]:
# Filtering based on the string column
print(gdf[gdf.integers == 2])

   integers strings
1         2       b


### Method 2:  Simple Rows

#### Filtering by Row Numbers

In [19]:
# Filter rows 0 to 2 (not inclusive of the third row with the index 2)
print(gdf[0:2])

   integers strings
0         1       a
1         2       b


### Method 3:  loc[rows, columns]

In [20]:
# The syntax is as follows loc[rows, columns] allowing you to choose rows and columns accordingly
# The example allows us to filter the first 3 rows (inclusive) of the column integers
print(gdf.loc[0:2, ['integers']])

   integers
0         1
1         2
2         3
